In [1]:
%reload_ext autoreload
%autoreload 2

## Algoritmos Evolutivos 1
### Carrera de Especialización en Inteligencia Artificial - FIUBA

### Trabajo Final

### 5º Bimestre 2025

### Grupo

| Autores               | E-mail                    | Nº SIU  |
|---------------------- |---------------------------|---------|
| Braian Desía          | b.desia@hotmail.com       | a1804   |

## MSTGAM-4trading

### 1. DESCRIPCIÓN DEL DESAFÍO

Se presenta la implementación de un **sistema de trading algorítmico** optimizado mediante un **Algoritmo Genético (GA)**, diseñado específicamente para la agregación y priorización de recomendaciones procedentes de múltiples estrategias de trading basadas en el paradigma de *Directional Changes* (DC) con distintos umbrales de detección de cambios direccionales.

El objetivo principal es desarrollar un marco de decisión robusto capaz de generar señales óptimas de inversión (buy, sell o hold) en mercados de acciones, combinando de forma inteligente las recomendaciones de ocho estrategias de trading DC-based, cada una perfilada bajo diez umbrales diferentes (θ). Este enfoque multi-estrategia y multi-umbral permite capturar una visión más completa y granular de la dinámica del mercado, superando las limitaciones de los enfoques tradicionales basados en tiempo físico fijo o en un único umbral DC. El Algoritmo Genético propuesto optimiza los pesos y prioridades de las sub-estrategias individuales, resolviendo conflictos entre señales contradictorias y maximizando el rendimiento ajustado al riesgo sin requerir suposiciones previas sobre el comportamiento del mercado.

### 1. IMPLEMENTACIÓN

#### 1.1 LIBRERÍAS

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import os

from main import DCTrader, Backtest, DCTracker
from trainer import GATrainer, PSOTrainer

#### 1.1 MODEL DEFINITION

In [3]:
from strategies import St1, St2, St3, St4, St5, St6, St7, St8

ticker = 'AAPL'

thresholds = [0.00098, 0.0022, 0.0048, 0.0072, 0.0098, 0.0122, 0.0155, 0.017, 0.02, 0.0255]
strategies = [St1, St2, St3, St4, St5, St6, St7, St8]

# Modo entrenamiento (precomputa estados)
trader = DCTrader(
    ticker=ticker,
    thresholds=thresholds,
    strategies=strategies,
    is_train=True,
    start_date='2016-01-01',
    end_date='2024-01-01'
)

print(f"Trader creado para {ticker}")
print(f"Thresholds: {thresholds}")
print(f"Datos cargados: {len(trader.prices)} días")

AAPL: 2012 loaded days | 2016-01-04 to 2023-12-29
Trader creado para AAPL
Thresholds: [0.00098, 0.0022, 0.0048, 0.0072, 0.0098, 0.0122, 0.0155, 0.017, 0.02, 0.0255]
Datos cargados: 2012 días


In [5]:
# import numpy as np
# n_strats = len(strategies)
# n_ths = len(thresholds)
# weights = np.ones((n_strats, n_ths)) / 70  # Igual peso en todos θ por estrategia
# results = trader.backtest(weights=weights)
# print(results)  # Debe dar SR >0 y trades

#### 1.1.1 ENTRENAMIENTO

In [ ]:
trader.fit()

Iniciando entrenamiento GA | Población: 150 | Generaciones: 50
Número total de pesos a optimizar: 70
Gen   1 | SR: -0.016 | RoR: -0.212 | ToR: 0.16402 | Trades: 330 | Fit: -0.163
Gen  10 | SR: -0.002 | RoR: -0.111 | ToR: 0.15755 | Trades: 317 | Fit: -0.124
Gen  20 | SR: 0.066 | RoR: 0.437 | ToR: 0.13668 | Trades: 275 | Fit: -0.007
Gen  30 | SR: 0.112 | RoR: 0.799 | ToR: 0.09990 | Trades: 201 | Fit: 0.124
Gen  40 | SR: 0.109 | RoR: 0.729 | ToR: 0.10040 | Trades: 202 | Fit: 0.124


#### 1.1 EVALUACIÓN

#### 1.1. CONCLUSIONES

#### 1.1.1 TO DO